In [31]:
# TRansformacion de imagenes (Segmentacion)

In [1]:
import gc
gc.collect()

7

# Segmentacion de datos

In [33]:
import cv2
import os
import numpy as np
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time

# Combinaciones (definidas completas)
combinaciones_no_casco = [
    ImageDataGenerator(horizontal_flip=True, shear_range=5),
    ImageDataGenerator(rotation_range=10, brightness_range=(1.2, 1.5)),
    ImageDataGenerator(rotation_range=15, zoom_range=[0.7, 1.4]),
    #ImageDataGenerator(brightness_range=(0.5, 0.8), rotation_range=15),
   # ImageDataGenerator(zoom_range=0.5),  # extra
]

combinaciones_casco = [
    ImageDataGenerator(rotation_range=30, horizontal_flip=True),
    ImageDataGenerator(rotation_range=10, brightness_range=(1.2, 1.5)),
    ImageDataGenerator(brightness_range=(0.5, 0.8), rotation_range=15),
    #ImageDataGenerator(rotation_range=15, vertical_flip=True),
    ImageDataGenerator(width_shift_range=0.2, shear_range=5),
    ImageDataGenerator(horizontal_flip=True, zoom_range=[0.7, 1.4]),
]

def cargar_imagenes_cv2(ruta_carpeta):
    imagenes = []
    archivos = os.listdir(ruta_carpeta)
    for archivo in archivos:
        img_path = os.path.join(ruta_carpeta, archivo)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (128, 128))
        imagenes.append((img, archivo))
    return imagenes

def guardar_imagenes_aumentadas(imagenes, datagens, ruta_salida, n_transform):
    os.makedirs(ruta_salida, exist_ok=True)
    total_imagenes = len(imagenes) * (1 + n_transform)  # original + aumentadas
    contador = 0
    tiempo_inicio = time.time()
    
    for idx, (imagen, nombre_archivo) in enumerate(imagenes):
        nombre_base = os.path.splitext(nombre_archivo)[0]
        
        # Guardar original
        img_bgr = cv2.cvtColor(imagen, cv2.COLOR_RGB2BGR)
        nombre_original = f"{nombre_base}_original.jpg"
        cv2.imwrite(os.path.join(ruta_salida, nombre_original), img_bgr)
        contador += 1
        
        # Mostrar progreso
        tiempo_actual = time.time()
        tiempo_transcurrido = tiempo_actual - tiempo_inicio
        tiempo_promedio = tiempo_transcurrido / contador
        tiempo_restante = tiempo_promedio * (total_imagenes - contador)
        progreso_pct = (contador / total_imagenes) * 100
        print(f"\rProcesando imagen {contador}/{total_imagenes} "
              f"({progreso_pct:.2f}%) - Tiempo estimado restante: {tiempo_restante:.1f} seg", end='')
        
        # Elegir aleatoriamente las transformaciones
        datagens_seleccionados = random.sample(datagens, n_transform)
        img_exp = np.expand_dims(imagen, 0)
        
        for i, datagen in enumerate(datagens_seleccionados):
            it = datagen.flow(img_exp, batch_size=1)
            imagen_aug = next(it)[0].astype(np.uint8)
            imagen_aug_bgr = cv2.cvtColor(imagen_aug, cv2.COLOR_RGB2BGR)
            nombre_nuevo = f"{nombre_base}_aug{i}.jpg"
            cv2.imwrite(os.path.join(ruta_salida, nombre_nuevo), imagen_aug_bgr)
            contador += 1
            
            # Actualizar progreso
            tiempo_actual = time.time()
            tiempo_transcurrido = tiempo_actual - tiempo_inicio
            tiempo_promedio = tiempo_transcurrido / contador
            tiempo_restante = tiempo_promedio * (total_imagenes - contador)
            progreso_pct = (contador / total_imagenes) * 100
            
            print(f"\rProcesando imagen {contador}/{total_imagenes} "
                  f"({progreso_pct:.2f}%) - Tiempo estimado restante: {tiempo_restante:.1f} seg", end='')
    print("\n✅ Proceso terminado.")

# Rutas
ruta_no_casco = r'C:\Users\Andres\Desktop\Data\Newalldata\Newdata\No_casco'
ruta_casco = r'C:\Users\Andres\Desktop\Data\Newalldata\Newdata\Casco'
ruta_salida_base = r'C:\Users\Andres\Desktop\Data\Newalldata\dataset_transformado'

ruta_no_casco_salida = os.path.join(ruta_salida_base, 'No_casco')
ruta_casco_salida = os.path.join(ruta_salida_base, 'Casco')

# Cargar imágenes
imagenes_no_casco = cargar_imagenes_cv2(ruta_no_casco)
imagenes_casco = cargar_imagenes_cv2(ruta_casco)

# Ejecutar
guardar_imagenes_aumentadas(imagenes_no_casco, combinaciones_no_casco, ruta_no_casco_salida, n_transform=3)
guardar_imagenes_aumentadas(imagenes_casco, combinaciones_casco, ruta_casco_salida, n_transform=5)

ModuleNotFoundError: No module named 'tensorflow.keras'

# Redimencionar y cargar datos

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

TAMANO_IMG = 128

# Usar ruta con doble backslash (\\) o con r"" crudo
ruta_dataset = "C:\\Users\\Andres\\Desktop\\Data\\Newalldata\\dataset_transformado"  # Carpeta que contiene Casco y No_casco

clases = {'No_casco': 0, 'Casco': 1}
imagenes = []
etiquetas = []

# Cargar imágenes
for clase, etiqueta in clases.items():
    carpeta = os.path.join(ruta_dataset, clase)
    if not os.path.isdir(carpeta):
        print(f"⚠️ Carpeta no encontrada: {carpeta}")
        continue

    for archivo in os.listdir(carpeta):
        ruta_imagen = os.path.join(carpeta, archivo)

        try:
            imagen = cv2.imread(ruta_imagen)
            if imagen is None:
                print(f"❌ No se pudo leer la imagen: {ruta_imagen}")
                continue

            imagen = cv2.resize(imagen, (TAMANO_IMG, TAMANO_IMG))
            imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
            imagenes.append(imagen)
            etiquetas.append(etiqueta)
        except Exception as e:
            print(f"⚠️ Error con {ruta_imagen}: {e}")

# Convertir listas a arrays
X = np.array(imagenes)
y = np.array(etiquetas)

# División de datos: 70% train, 15% val, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

print("Train:", X_train.shape)
print("Val:", X_val.shape)
print("Test:", X_test.shape)

Train: (31441, 128, 128, 3)
Val: (6737, 128, 128, 3)
Test: (6738, 128, 128, 3)


# Normalizar datos de entrada

In [2]:

import numpy as np

#X = np.array(X).astype(np.float32) / 255.0

X_train = X_train.astype(np.float32) / 255.0
X_val = X_val.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0


In [3]:
X.shape


(44916, 128, 128, 3)

In [ ]:
# Creacion de modelos

In [54]:
import tensorflow as tf

modeloDenso = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 128, 3)),
  tf.keras.layers.Dense(150, activation='relu'),
  tf.keras.layers.Dense(150, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

modeloCNN = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

modeloCNN2 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(250, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

c:\Users\Andres\miniconda3\envs\myenvironment\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\Andres\miniconda3\envs\myenvironment\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
#Compilar modelos. Usar crossentropy binario ya que tenemos solo 2 opciones (perro o gato)
modeloDenso.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

modeloCNN.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

modeloCNN2.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

tensorboardDenso = TensorBoard(log_dir='logs/denso')
modeloDenso.fit(
    X_train, y_train,
    batch_size=32,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[tensorboardDenso]
)

Epoch 1/100
237/237 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.6770 - loss: 1.6816 - val_accuracy: 0.7392 - val_loss: 0.5155
Epoch 2/100
237/237 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7399 - loss: 0.5125 - val_accuracy: 0.7707 - val_loss: 0.4509
Epoch 3/100
237/237 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7566 - loss: 0.4751 - val_accuracy: 0.7922 - val_loss: 0.4192
Epoch 4/100
237/237 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7736 - loss: 0.4391 - val_accuracy: 0.7965 - val_loss: 0.4017
Epoch 5/100
237/237 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.7834 - loss: 0.4228 - val_accuracy: 0.7811 - val_loss: 0.4162
Epoch 6/100
237/237 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7876 - loss: 0.4148 - val_accuracy: 0.7485 - val_loss: 0.4433
Epoch 7/100
237/237 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.7973 - loss: 0.4119 - val_accuracy: 0.7626 - val_loss: 0.4490
Epoch 8/100
237/237 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.7794 - loss: 0.4221 - 

In [ ]:
#Cargar la extension de tensorboard de colab
%load_ext tensorboard
#Ejecutar tensorboard e indicarle que lea la carpeta "logs"
%tensorboard --logdir logs

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

# Para modeloCNN
tensorboardCNN = TensorBoard(log_dir='logs/cnn')
modeloCNN.fit(
    X_train, y_train,
    batch_size=32,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[tensorboardCNN]
)


Epoch 1/100
 69/237 ━━━━━━━━━━━━━━━━━━━━ 17s 106ms/step - accuracy: 0.8738 - loss: 0.2788

KeyboardInterrupt: 

In [3]:

# Para modeloCNN2
tensorboardCNN2 = TensorBoard(log_dir='logs/cnn2')
modeloCNN2.fit(
    X_train, y_train,
    batch_size=32,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[tensorboardCNN2]
)

NameError: name 'TensorBoard' is not defined

In [4]:
from tensorflow.keras import layers, models

modeloCNN2_opt = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

c:\Users\Andres\miniconda3\envs\tf-gpu-env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
modeloCNN2_opt.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy'])

In [ ]:
# Callback para monitorear con TensorBoard (opcional)
from tensorflow.keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='logs/cnn2_opt')

# Entrenamiento del modelo
modeloCNN2_opt.fit(
    X_train, y_train,
    batch_size=32,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[tensorboard],
    verbose=1
)

Epoch 1/100
983/983 ━━━━━━━━━━━━━━━━━━━━ 315s 317ms/step - accuracy: 0.8375 - loss: 0.5845 - val_accuracy: 0.8841 - val_loss: 0.2890
Epoch 2/100
983/983 ━━━━━━━━━━━━━━━━━━━━ 307s 312ms/step - accuracy: 0.9253 - loss: 0.1860 - val_accuracy: 0.8859 - val_loss: 0.2840
Epoch 3/100
983/983 ━━━━━━━━━━━━━━━━━━━━ 309s 314ms/step - accuracy: 0.9520 - loss: 0.1307 - val_accuracy: 0.8140 - val_loss: 0.4885
Epoch 4/100
983/983 ━━━━━━━━━━━━━━━━━━━━ 305s 310ms/step - accuracy: 0.9616 - loss: 0.1030 - val_accuracy: 0.9480 - val_loss: 0.1534
Epoch 5/100
983/983 ━━━━━━━━━━━━━━━━━━━━ 300s 305ms/step - accuracy: 0.9701 - loss: 0.0784 - val_accuracy: 0.9633 - val_loss: 0.1030
Epoch 6/100
983/983 ━━━━━━━━━━━━━━━━━━━━ 297s 302ms/step - accuracy: 0.9778 - loss: 0.0599 - val_accuracy: 0.9592 - val_loss: 0.1128
Epoch 7/100
983/983 ━━━━━━━━━━━━━━━━━━━━ 302s 307ms/step - accuracy: 0.9789 - loss: 0.0602 - val_accuracy: 0.9575 - val_loss: 0.1693
Epoch 8/100
983/983 ━━━━━━━━━━━━━━━━━━━━ 311s 316ms/step - accuracy: 

In [ ]:
#Cargar la extension de tensorboard de colab
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
#Ejecutar tensorboard e indicarle que lea la carpeta "logs"
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 2400), started 7:54:02 ago. (Use '!kill 2400' to kill it.)

In [ ]:
# Guardar modelo completo
modeloCNN2_opt.save("modeloCNN2_opt.h5")